In [1]:
from pymongo import MongoClient
import os
from datetime import datetime
import pandas as pd

In [2]:
%run -i "utilities.py"

In [3]:
def get_stemmed_tokens(filename) :
    text = get_text(filename)
    text = remove_stopwords(text)
    tokens = stem_file(text)
    return tokens

In [4]:
client = MongoClient()
db = client["search_engine"]

In [5]:
path = "/home/ironman/Documents/Coding/Search Engine/Files/"
def update_index(path) :
    for file in os.listdir(path) :
        #check if not updated
        find_file = pd.DataFrame(db.updated_logs.find({"filename" : file}))
        
        if  find_file.empty : #or timestamp greater than threshold
            tokens, word_frequency, stem_frequency = get_stemmed_tokens(path + file)
            for token in tokens :
                
                word = token[0]
                stemmed_word = token[1]
                line_number = token[2]
                db.word_index.insert_one({"word" : word, "stem" : stemmed_word, "filename" : file, "line_number" : line_number})
            
            for word in word_frequency :
                db.word_frequency_logs.insert_one({"word" : word, "term_frequency" : word_frequency[word], "filename" : file})
            for stem in stem_frequency :
                db.stem_frequency_logs.insert_one({"stem" : stem, "term_frequency" : stem_frequency[stem], "filename" : file})
            #mark as updated with last modified timestamp
            db.updated_logs.insert_one({"filename" : file, "time" : datetime.now()})

In [6]:
update_index(path)

In [7]:
data = pd.DataFrame(db.word_index.find({}))

In [8]:
data

,_id,word,stem,filename,line_number
0,5fdcf1fc7e33483c1f7e8790,में,में,40.txt,189
1,5fdcf1fd7e33483c1f7e8791,नहीं,नह,40.txt,56
2,5fdcf1fd7e33483c1f7e8792,पर,पर,40.txt,76
3,5fdcf1fd7e33483c1f7e8793,क्षेत्रों,क्षेत्र,40.txt,93
4,5fdcf1fd7e33483c1f7e8794,रचनाएँ,रचनाएँ,40.txt,338
...,...,...,...,...,...
88356,5fdcf25a7e33483c1f812e6f,या,या,37.txt,407
88357,5fdcf25a7e33483c1f812e70,राज्य,राज्य,37.txt,149
88358,5fdcf25a7e33483c1f812e71,कि,कि,37.txt,236
88359,5fdcf25a7e33483c1f812e72,हो,हो,37.txt,101


In [9]:
datadf = pd.DataFrame(db.updated_logs.find())

In [10]:
datadf.sort_values(by = "filename")

,_id,filename,time
30,5fdcf2117e33483c1f7f1177,1.txt,2020-12-18 23:46:49.438
58,5fdcf2207e33483c1f7f7f36,10.txt,2020-12-18 23:47:04.215
151,5fdcf2527e33483c1f80f1f1,100.txt,2020-12-18 23:47:54.426
126,5fdcf2437e33483c1f808790,101.txt,2020-12-18 23:47:39.792
83,5fdcf22c7e33483c1f7fdab1,102.txt,2020-12-18 23:47:16.603
...,...,...,...
100,5fdcf2367e33483c1f802496,95.txt,2020-12-18 23:47:26.359
79,5fdcf22a7e33483c1f7fcce7,96.txt,2020-12-18 23:47:14.800
155,5fdcf2557e33483c1f810910,97.txt,2020-12-18 23:47:57.597
104,5fdcf2387e33483c1f8034cd,98.txt,2020-12-18 23:47:28.569
